In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/book-recommendation-data/Ratings.csv
/kaggle/input/book-recommendation-data/Users.csv
/kaggle/input/book-recommendation-data/Books.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings('ignore')

In [3]:
books = pd.read_csv('/kaggle/input/book-recommendation-data/Books.csv')
ratings = pd.read_csv('/kaggle/input/book-recommendation-data/Ratings.csv')
users = pd.read_csv('/kaggle/input/book-recommendation-data/Users.csv')


In [4]:
books.shape,users.shape,ratings.shape

((271360, 8), (278858, 3), (1149780, 3))

In [5]:
books.isna().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [6]:
users.isna().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [7]:
ratings.isna().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [8]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [9]:
books.ISBN.nunique()

271360

In [10]:
books.duplicated().sum()

0

In [11]:
ratings.duplicated().sum()

0

In [12]:
users.duplicated().sum()

0

# Model Building of popularity based recommender system:

In [13]:
ratings_books_name = ratings.merge(books,on='ISBN')

In [14]:
ratings_books_name.shape,ratings.shape,books.shape

((1031136, 10), (1149780, 3), (271360, 8))

* Ratings have some other books that are not present in books dataset.
* Lets find out those books which ave ratings given by more than 250 users..

In [15]:
num_rating_df = pd.DataFrame(ratings_books_name.groupby(by='Book-Title').count()['Book-Rating'])

In [16]:
num_rating_df.rename(columns={'Book-Rating':'Num_Rating'},inplace=True)

In [17]:
num_rating_df.head()

,Num_Rating
Book-Title,
"A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)",4
Always Have Popsicles,1
Apple Magic (The Collector's series),1
"Ask Lily (Young Women of Faith: Lily Series, Book 5)",1
Beyond IBM: Leadership Marketing and Finance for the 1990s,1


In [18]:
avg_rating_df = pd.DataFrame(ratings_books_name.groupby(by='Book-Title')['Book-Rating'].mean())
avg_rating_df.rename(columns={'Book-Rating':'Avg_Rating'},inplace=True)
avg_rating_df.head()

,Avg_Rating
Book-Title,
"A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)",2.25
Always Have Popsicles,0.00
Apple Magic (The Collector's series),0.00
"Ask Lily (Young Women of Faith: Lily Series, Book 5)",8.00
Beyond IBM: Leadership Marketing and Finance for the 1990s,0.00


In [19]:
popularity_df = num_rating_df.merge(avg_rating_df,on='Book-Title')

In [20]:
popular_df = popularity_df[popularity_df['Num_Rating']>250].sort_values('Avg_Rating',ascending=False).head(50)

In [21]:
popular_df = popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')

In [22]:
popular_df.head()

,Book-Title,Num_Rating,Avg_Rating,ISBN,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804,0439136350,J. K. Rowling,1999,Scholastic,http://images.amazon.com/images/P/0439136350.0...,http://images.amazon.com/images/P/0439136350.0...,http://images.amazon.com/images/P/0439136350.0...
3,Harry Potter and the Goblet of Fire (Book 4),387,5.824289,0439139597,J. K. Rowling,2000,Scholastic,http://images.amazon.com/images/P/0439139597.0...,http://images.amazon.com/images/P/0439139597.0...,http://images.amazon.com/images/P/0439139597.0...
5,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410,0590353403,J. K. Rowling,1998,Scholastic,http://images.amazon.com/images/P/0590353403.0...,http://images.amazon.com/images/P/0590353403.0...,http://images.amazon.com/images/P/0590353403.0...
9,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441,043935806X,J. K. Rowling,2003,Scholastic,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...
13,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453,0439064872,J. K. Rowling,2000,Scholastic,http://images.amazon.com/images/P/0439064872.0...,http://images.amazon.com/images/P/0439064872.0...,http://images.amazon.com/images/P/0439064872.0...


In [23]:
popular_df['Publication_Decade'] = (popular_df['Year-Of-Publication'] // 10) * 10

In [24]:
popular_df['Publisher_Popularity'] = popular_df.groupby('Publisher')['Book-Title'].transform('count')

popular_df = ratings['Book-Rating']
popular_df['Publisher_Avg_Rating'] = popular_df.groupby('Publisher')['Book-Rating'].transform('mean')

In [25]:
popular_df.columns

Index(['Book-Title', 'Num_Rating', 'Avg_Rating', 'ISBN', 'Book-Author',
       'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M',
       'Image-URL-L', 'Publication_Decade', 'Publisher_Popularity'],
      dtype='object')

In [26]:
popular_df = popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Num_Rating','Avg_Rating','Book-Author_x','Image-URL-M_x','Publication_Decade','Publisher_Popularity']]

In [27]:
popular_df.shape

(50, 7)

# Collaborative filtering Based Recommender System

In [28]:
x = ratings_books_name.groupby(by='User-ID').count()['Book-Rating'] 
educated = x[x>200].index

In [29]:
filtered_rating = ratings_books_name[ratings_books_name['User-ID'].isin(educated)]

In [30]:
y = filtered_rating.groupby(by='Book-Title').count()['Book-Rating']
famous_books = y[y>=50].index

In [31]:
famous_books

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Day Late and a Dollar Short', 'A Fine Balance',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='Book-Title', length=706)

In [32]:
final_rating = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [33]:
final_rating.duplicated().sum()

0

In [34]:
pivot_table = final_rating.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [35]:
pivot_table.fillna(0,inplace=True)

In [36]:
from sklearn.metrics.pairwise import cosine_similarity

In [37]:
similarity_score = cosine_similarity(pivot_table)

In [38]:
similarity_score.shape

(706, 706)

In [39]:
def recommend(book_name):
    index = np.where(pivot_table.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_score[index])),key=lambda x:x[1],reverse=True)[1:6]
    
    for i in similar_items:
        print(pivot_table.index[i[0]])


In [40]:
recommend('Message in a Bottle')

Nights in Rodanthe
The Mulberry Tree
A Walk to Remember
River's End
Nightmares &amp; Dreamscapes


In [41]:
recommend('1984')

Animal Farm
The Handmaid's Tale
Brave New World
The Vampire Lestat (Vampire Chronicles, Book II)
The Hours : A Novel


In [42]:
recommend('Naked')

Me Talk Pretty One Day
Bastard Out of Carolina
Brave New World
Bridget Jones : The Edge of Reason
1984


In [43]:
recommend('Zoya')

Fine Things
Secrets
Kaleidoscope
Exclusive
Five Days in Paris


# Nearest Neighbors Algorithm Used

In [44]:
books.drop(columns=['Image-URL-M','Image-URL-L','Image-URL-S'],inplace=True)

In [45]:
books.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial


In [46]:
books.rename(columns={'Book-Title':'title','Book-Author':'author','Year-Of-Publication':'year','Publication':'publication'},inplace=True)

In [47]:
books.head(3)

,ISBN,title,author,year,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial


In [48]:
users.rename(columns={'User-ID':'user_id','Location':'location','Age':'age'},inplace=True)

In [49]:
ratings.rename(columns={'User-ID':'user_id','Book-Rating':'rating'},inplace=True)

In [50]:
ratings.head(3)

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0


In [51]:
books.shape

(271360, 5)

In [52]:
ratings.shape

(1149780, 3)

In [53]:
users.shape

(278858, 3)

In [54]:
a = ratings['user_id'].value_counts()

In [55]:
b = a[a>200]

In [56]:
b = b.index

In [57]:
ratings1 = ratings[ratings['user_id'].isin(b)]

In [58]:
ratings1.shape

(526356, 3)

In [59]:
ratings_with_books = ratings1.merge(books)

In [60]:
ratings_with_books.shape

(487671, 7)

In [61]:
number_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [62]:
number_rating.rename(columns={'rating':'No_of_rating'},inplace=True)

In [63]:
final_rating = ratings_with_books.merge(number_rating)

In [64]:
final_rating_data = final_rating[final_rating['No_of_rating'] >= 50]

In [65]:
final_rating_data.shape

(61853, 8)

In [66]:
final_rating_data.drop_duplicates(['user_id','title'],inplace=True)

In [67]:
final_rating_data

,user_id,ISBN,rating,title,author,year,Publisher,No_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
...,...,...,...,...,...,...,...,...
236701,255489,0553579983,7,And Then You Die,Iris Johansen,1998,Bantam,50
236702,256407,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,50
236703,257204,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,50
236704,261829,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,50


In [68]:
final_rating_data.shape

(59850, 8)

In [69]:
book_pivot = final_rating_data.pivot_table(columns='user_id',index='title',values='rating')

In [70]:
book_pivot.fillna(0,inplace=True)

In [71]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot)

In [72]:
type(book_sparse)

scipy.sparse._csr.csr_matrix

### Modelling

In [73]:
from sklearn.neighbors import NearestNeighbors

In [74]:
model = NearestNeighbors(algorithm='brute')

In [75]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [76]:
distances , suggestions = model.kneighbors(book_pivot.iloc[240,:].values.reshape(1,-1),n_neighbors=6)

In [77]:
suggestions

array([[240, 238, 237, 241, 239, 184]])

In [78]:
for i in range(len(suggestions)):
    print(book_pivot.index[suggestions[i]])

Index(['Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Sorcerer's Stone (Book 1)',
       'Harry Potter and the Order of the Phoenix (Book 5)', 'Exclusive'],
      dtype='object', name='title')


In [79]:
def recommend(book_name):
    book_id = np.where(book_pivot.index==book_name)[0][0]
    distances , suggestions = model.kneighbors(book_pivot.iloc[book_id,:]\
                                               .values.reshape(1,-1),n_neighbors=6)
    for i in range(len(suggestions)):
        print(book_pivot.index[suggestions[i]][1:])

In [80]:
recommend('Animal Farm')

Index(['Exclusive', 'Jacob Have I Loved', 'Second Nature', 'Pleading Guilty',
       'No Safe Place'],
      dtype='object', name='title')


In [81]:
recommend('Zoya')

Index(['Fine Things', 'Exclusive', 'Secrets', 'The Cradle Will Fall',
       'Kaleidoscope'],
      dtype='object', name='title')


In [82]:
recommend('Naked')

Index(['No Safe Place', 'Deck the Halls (Holiday Classics)',
       'Long After Midnight', 'Exclusive', 'Lake Wobegon days'],
      dtype='object', name='title')
